In [29]:
import os
import sys

In [37]:
ROOT = os.path.join(os.pardir)
MODEL_DIR = os.path.join(ROOT, 'models')

sys.path.append(os.path.join(ROOT, "src"))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
IMG_SIZE = 224
N_CLASSES = 196
LN_RATE = 0.001

# Create Model

## ResNet50

In [41]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import (
    Dense, Activation, Flatten, Dropout
)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

In [12]:
base_model = ResNet50(weights='imagenet', 
                 include_top=False,
                 input_shape=(IMG_SIZE, IMG_SIZE, 3),
                 pooling='avg')

In [18]:
base_model.input

<tf.Tensor 'input_3:0' shape=(?, 224, 224, 3) dtype=float32>

In [14]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [19]:
# Freeze pretrained layers
for layer in base_model.layers:
    layer.trainable = False
    
x = base_model.output
x = Flatten()(x)
predictions = Dense(N_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [20]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [43]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(lr=LN_RATE),
    # what about precision and recall?
    metrics=['acc', Precision(), Recall()]
)

What metrics to keep track of?

In [40]:
model.save(os.path.join(MODEL_DIR, 'model.h5'))

In [39]:
model.save_weights(os.path.join(MODEL_DIR, 'model.h5'))